---

# Assignment 2

Welcome to the second assignment! Here you will implement Matrix Factorization. We will use part of the MovieLens 20M dataset.

You will write and execute your code in Python using this Jupyter Notebook.

**PREREQUISITE:** Download the MovieLens 20M dataset from <https://grouplens.org/datasets/movielens/20m/>. Extract the contents and look up `ratings.csv`, which is what we'll be working with.

**TASK:** Your job is to *fill in the missing code* only. The place to enter your code is clearly marked with comments.

**SUBMISSION:** You will submit this Notebook via TUWEL.

**GRADING:** We will test whether your code produces the expected output. Therefore hidden tests will compare results of the standard solution with yours (based on the whole dataset - multiple, randomly selected inputs - accuracy of the sulution must be within two decimal places).

Note that there is a variable `DEBUG` set to `True` for debugging/testing purposes. **For this assignment all tests will be based on the truncated dataset.**

## Preparation

Import some basic python modules and set some print formatting options.

In [4]:
import csv
import pandas as pd
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp
import sys
import math

np.set_printoptions(threshold=500, precision=4)
pd.options.display.max_seq_items = 100

### Read the data

We will work with a *truncated* version of the MovieLens 20M dataset, containing up to 10000 users and 1000 movies. 

In [5]:
DEBUG = True
data_location = '../../data/ratings.csv' ## SOME LOCATION OF YOUR CHOICE

In [6]:
ratings_raw = pd.read_csv(data_location)
if DEBUG:
    ratings_raw = ratings_raw[ (ratings_raw['userId'] < 10000) & (ratings_raw['movieId'] < 1000) ]

Let's see how the data looks like.

In [7]:
display(ratings_raw.head())

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


### Preprocess the data

We make sure that movies and users have consecutive indexes starting from 0. We also drop the timestamp column.

The resulting "cleaned" data are stored in `ratings`.

In [8]:
movieIds = ratings_raw.movieId.unique()
movieIds.sort()
userIds = ratings_raw.userId.unique()
userIds.sort()

m = userIds.size
n = movieIds.size
numRatings = len(ratings_raw)

print ("There are", m, "users,", n, "items and", numRatings, "ratings.")

## movies and users should have consecutive indexes starting from 0
movieId_to_movieIDX = dict(zip(movieIds, range(0, movieIds.size)))
movieIDX_to_movieId = dict(zip(range(0, movieIds.size), movieIds))

userId_to_userIDX = dict(zip(userIds, range(0, userIds.size )))
userIDX_to_userId = dict(zip(range(0, userIds.size), userIds))

## drop timestamps
ratings = pd.concat([ratings_raw['userId'].map(userId_to_userIDX), ratings_raw['movieId'].map(movieId_to_movieIDX), ratings_raw['rating']], axis=1)
ratings.columns = ['user', 'item', 'rating']

display(ratings.head())

There are 9924 users, 968 items and 394638 ratings.


,user,item,rating
0,0,1,3.5
1,0,28,3.5
2,0,31,3.5
3,0,46,3.5
4,0,49,3.5


### Create the Ratings Matrix

We will convert the `ratings` `DataFrame` into a **Ratings Matrix**. Because it is very sparse, we will use the `scipy.sparse` module to efficiently store and access it.

Specifically, we will create the ratings matrix `R` stored in the Compressed Sparse Row format (`csr_matrix`).

In [9]:
R = sp.csr_matrix((ratings.rating, (ratings.user, ratings.item)))

m = R.shape[0]
n = R.shape[1]
n_ratings = R.count_nonzero()

## The fun starts here!

Import additional modules to simplify some operations, and define some helper functions.

In [10]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import mean_squared_error


def get_one_hot(targets):
    lb = pp.LabelBinarizer(sparse_output=True)
    lb.fit(targets.reshape(-1))
    return lb.transform(targets)


def to_Xy_format(R):
    n_users = R.shape[0]
    n_items = R.shape[1]

    users, items = R.nonzero()
    n_ratings = users.size

    Xu = get_one_hot(users)
    Xi = get_one_hot(items)

    R = sp.csr_matrix(R)
    y = R.data
    X = sp.hstack([Xu, Xi])

    return X, y, n_users, n_items


def to_R_format(X, y, n_users, n_items):
    Xu = X.tocsc()[:, :n_users]
    Xi = X.tocsc()[:, n_users:]

    Xu = Xu.tocsr()
    Xi = Xi.tocsr()

    R_rec = sp.coo_matrix( (y, (Xu.indices, Xi.indices)), shape=(n_users, n_items) )

    return R_rec.tocsr()


def to_UI_arrays(X, n_users, n_items):
    Xu = X.tocsc()[:, :n_users]
    Xi = X.tocsc()[:, n_users:]

    U = Xu.argmax(axis=1).A1
    I = Xi.argmax(axis=1).A1
    
    return U, I

While the ratings matrix `R` is very intuitive, sometimes it is more convenient to have a view of one rating (training example) at a time in terms of the `X` and `y` arrays.

The `X` array has $m+n$ columns, where the first $m$ one-hot encode the user, and the other $n$ the item. So a row of `X` has exactly two `1`s and all other entries are zero. Array `X` has as many rows as the number of ratings. So, the shape of `X` is `(n_ratings, n_users+n_items)`. (Actually `X` is stored as a sparse matrix, so the corresponding dense matrix has that shape.)

The `y` array has a single column, and each row contains the rating for user, item pair indicated by the corresponding row in the `X` array. So, the shape of `y` is `(n_ratings,)`.

The above functions convert between the two formats. Note that to convert from the `Xy` format you need to know the number of users and items, `n_users`, `n_items` respectively.

In [11]:
X, y, n_users, n_items = to_Xy_format(R)

## Build the MatrixFactorization class

In the following, we will build functionality into a `MatrixFactorization` class, which inherits from the [scikit-learn](http://scikit-learn.org) classes `BaseEstimator` and `RegressorMixin`. The reason we follow the scikit-learn API is that we may want to use some of the functionality offered, such as hyperparameter tuning; but not in this assignment. If you're interested, refer to <http://scikit-learn.org/stable/developers/contributing.html#rolling-your-own-estimator> for more on the scikit-learn API.

The model uses the following hyperparameters:

- `k` is the number of factors
- `eta` is the learning rate $\eta$ of SGD
- `lam` is the regularization strength $\lambda$
- `n_epochs` is the number of epochs to run SGD
- `s_batch` is the size of the batch in the mini-batch SGD (bonus point)
- `w_average` indicates if the overall average rating is used in the baseline prediction
- `w_biases` indicates if user and item biases are used in the baseline prediction
- `rnd_mean` and `rnd_std` are the mean and std. deviation of the Normal distribution used for initializing the model parameters.

In [12]:
class MatrixFactorization(BaseEstimator, RegressorMixin):
    
    def __init__(self, k = 5, eta = 0.002, lam = 0., n_epochs = 5, s_batch = 1, w_average = True, w_biases = True, rnd_mean = 0, rnd_std = 0.1):
        self.k = k
        self.eta = eta
        self.lam = lam
        self.n_epochs = n_epochs
        self.s_batch = s_batch
        self.w_average = w_average
        self.w_biases = w_biases
        self.rnd_mean = rnd_mean
        self.rnd_std = rnd_std

        
class LossObserver():
    def __init__(self, loss=sys.maxsize):
        self.loss = loss
        
        
VERBOSE = True

### Prepare for training --- TO EDIT

The model you will build should make predictions as:

$$ \hat{r}_{ui} = \mu + b_u + b_i + p_u^\intercal q_i $$

where:

- $\mu$ is the overall average rating and should be stored in `self.mu_`;
- $b_u$ is a user bias; all user biases should be stored in array `self.bu_` of shape `(n_users, )`;
- $b_i$ is a item bias; all item biases should be stored in array `self.bi_` of shape `(n_items, )`;
- $p_u$ is the feature vector of a user; all user feature vectors should be stored in array `self.P_` of shape `(n_users, k)`;
- $q_i$ is the feature vector of an item; all item feature vectors should be stored in array `self.Q_` of shape `(n_items, k)`.

Note that variables depending on the input data typically have a trailing underscore, as in `self.mu_`, and are computed in the `fit` method.
In contrast, variables that do not depend on the input data but on the model, such as hyperparameters, do not have a trailing underscore and are set in the `__init__` method.

** TASK: ** The `fit_init` method below should:
1. compute the overall average rating `self.mu_`, and
2. initialize the four arrays (`self.bu_`, `self.bi_`, `self.P_`, `self.Q_`) with values selected at random from a Normal distribution with mean `self.rnd_mean` and standard deviation `self.rnd_std`.

In [13]:
def fit_init(self, X, y, n_users, n_items):
    X, y = check_X_y(X, y, accept_sparse=True)

    self.n_users_ = n_users
    self.n_items_ = n_items
    self.n_ratings_ = X.shape[0]

    self.X_ = X
    self.y_ = y
    np.random.seed(1)
    
    # YOUR CODE HERE
    self.mu_ = np.mean(y)
    self.P_ = np.random.normal(self.rnd_mean, self.rnd_std, n_users * self.k)
    self.P_.shape = (n_users, self.k)
    self.Q_ = np.random.normal(self.rnd_mean, self.rnd_std, n_items * self.k)
    self.Q_.shape = (n_items, self.k)
    self.bu_ = np.random.normal(self.rnd_mean, self.rnd_std, n_users)
    self.bi_ = np.random.normal(self.rnd_mean, self.rnd_std, n_items)
        
    ## random shuffle the training data
    self.X_, self.y_ = shuffle(self.X_, self.y_)
    
    return self

Include the above function in the `MatrixFactorization` class.

In [14]:
MatrixFactorization.fit_init = fit_init

In [15]:
if DEBUG:
    mf = MatrixFactorization()

    mf.fit_init(X, y, n_users, n_items)

    print("mf.mu_ =", mf.mu_)

    print()
    print("parameters before training:")
    print("mf.P_ =", mf.P_)
    print("mf.Q_ =", mf.Q_)

mf.mu_ = 3.538466898778121

parameters before training:
mf.P_ = [[ 0.1624 -0.0612 -0.0528 -0.1073  0.0865]
 [-0.2302  0.1745 -0.0761  0.0319 -0.0249]
 [ 0.1462 -0.206  -0.0322 -0.0384  0.1134]
 ...
 [ 0.0736 -0.0751 -0.0063 -0.02    0.008 ]
 [-0.0331  0.1172 -0.0343 -0.0628 -0.0425]
 [ 0.0315  0.0229  0.0233 -0.0419 -0.0543]]
mf.Q_ = [[ 0.1182 -0.1105  0.0121  0.0044  0.0354]
 [ 0.0142  0.1286  0.024   0.1653 -0.0519]
 [ 0.0515 -0.0941 -0.1337  0.0395 -0.0346]
 ...
 [ 0.0525 -0.0582 -0.1166  0.0404 -0.036 ]
 [ 0.0518  0.0134 -0.1122  0.0342  0.006 ]
 [-0.0915 -0.025   0.1451 -0.1112 -0.1196]]


**DEBUG:** The previous cell should output:
```
mf.mu_ = 3.538466898778121

parameters before training:
mf.P_ = [[ 0.1624 -0.0612 -0.0528 -0.1073  0.0865]
 [-0.2302  0.1745 -0.0761  0.0319 -0.0249]
 [ 0.1462 -0.206  -0.0322 -0.0384  0.1134]
 ...
 [ 0.0736 -0.0751 -0.0063 -0.02    0.008 ]
 [-0.0331  0.1172 -0.0343 -0.0628 -0.0425]
 [ 0.0315  0.0229  0.0233 -0.0419 -0.0543]]
mf.Q_ = [[ 0.1182 -0.1105  0.0121  0.0044  0.0354]
 [ 0.0142  0.1286  0.024   0.1653 -0.0519]
 [ 0.0515 -0.0941 -0.1337  0.0395 -0.0346]
 ...
 [ 0.0525 -0.0582 -0.1166  0.0404 -0.036 ]
 [ 0.0518  0.0134 -0.1122  0.0342  0.006 ]
 [-0.0915 -0.025   0.1451 -0.1112 -0.1196]]
```

## Implement Stochastic Gradient Descent --- TO EDIT

The full prediction model is:
$$ \hat{r}_{ui} = \mu + b_u + b_i + p_u^\intercal q_i .$$

** TASK: ** The `fit_sgd` method should do the following for each rating $r_{ui}$:

1. Compute the prediction and store it in variable `prediction`.
    - If `self.w_average` and `self.w_biases` are both `True` then `prediction` should be computed as $\mu + b_u + b_i + p_u^\intercal q_i$.
    - If one of the flags `self.w_average`, `self.w_biases` are `False`, then the baseline prediction should be appropriately changed. For example, if they are both false then `prediction` should simply be computed as $p_u^\intercal q_i$.

2. Compute the prediction error $e_{ui}$ and store it in variable `err`.

3. Perform a step of SGD. That is implement the update rules:

$$p_u \gets p_u + \eta \cdot (e_{ui} \cdot q_i - \lambda \cdot p_u)$$
$$q_i \gets q_i + \eta \cdot (e_{ui} \cdot p_u - \lambda \cdot q_i)$$
$$b_u \gets b_u + \eta \cdot (e_{ui} - \lambda \cdot b_u)$$
$$b_i \gets b_i + \eta \cdot (e_{ui} - \lambda \cdot b_i)$$

Make sure you only update the biases if `self.w_biases` is `True`.

The `fit` method is just a wrapper for `fit_sgd` and `fit_mgd` (the last is bonus).

In [16]:
obs = LossObserver()

def fit_sgd(self): ## stochastic gradient descent
    
    U, I = to_UI_arrays(self.X_, self.n_users_, self.n_items_)
    
    
    if VERBOSE:
        print("start of training")

    for epoch in range(self.n_epochs):

        epoch_loss = 0.

        for index in range(self.y_.shape[0]):
            u = U[index]
            i = I[index]
            r_ui = self.y_[index]
            
            # YOUR CODE HERE
            prediction = self.P_[u].T.dot(self.Q_[i])
            if self.w_average:
                prediction += self.mu_
            if self.w_biases:
                prediction += self.bu_[u] + self.bi_[i]
            
            err = r_ui - prediction
            self.P_[u] += self.eta * (err * self.Q_[i] - self.lam * self.P_[u])
            self.Q_[i] += self.eta * (err * self.P_[u] - self.lam * self.Q_[i])
            if self.w_biases:
                self.bu_[u] += self.eta * (err - self.lam * self.bu_[u])
                self.bi_[i] += self.eta * (err - self.lam * self.bi_[i])
            
            epoch_loss += err * err

        ## epoch is done
        epoch_loss /= self.n_ratings_
        obs.loss = epoch_loss
        if VERBOSE:
            print("epoch", epoch, "loss", epoch_loss)

    return self


def fit(self, X, y, n_users, n_items):
    self.fit_init(X, y, n_users, n_items)
    
    if VERBOSE:
        print(self.get_params())

    if self.s_batch == 1: ## stochastic gradient descent
        self.fit_sgd()
    else: ## mini-batch stochastic gradient descent -- BONUS point
        self.fit_mgd()

    return self


# For more info about fit_mgt see Section BONUS Point
def fit_mgd(self): ## minibatch gradient descent, vectorized        

    if VERBOSE:
        print("start training")

    print(self.n_users_, self.n_items_)
    print("X_", self.X_.shape)

    print("self.P_", self.P_.shape)
    
    for epoch in range(self.n_epochs):

        epoch_loss = 0.

        for XB, yB in iterate_minibatches(self.X_, self.y_, self.s_batch):
            XB_U = XB[:,0:self.n_users_]
            XB_I = XB[:,self.n_users_:]
            PB = self.P_.dot(XB_U).T
            QB = self.Q_.dot(XB_I).T
            # prediction = PB.T.dot(QB)
            prediction = QB.T.dot(PB)
            print("prediction", prediction.shape)
            if self.w_average:
                prediction += self.mu_
            if self.w_biases:
                prediction += self.bu_.dot(XB) + self.bi_.dot(yB)
            err = self.y_.dot(yB) - prediction
            print("err", err.shape)
            self.P_ += self.eta * (err.T.dot(QB) - self.lam * PB)
            self.Q_ += self.eta * (err.T.dot(PB) - self.lam * QB)
            if self.w_biases:
               self.bu_ += self.eta * (err - self.lam * self.bu_.dot(XB))
               self.bi_ += self.eta * (err - self.lam * self.bi_.dot(yB))
            print("epoch_loss", epoch_loss)
            epoch_loss += err.T.dot(err)

        epoch_loss /= self.n_ratings_
        obs.loss = epoch_loss
        
        if VERBOSE:
            print("epoch", epoch, "loss", epoch_loss)

    return self

MatrixFactorization.fit_mgd = fit_mgd

# helper for fit_mgt (BONUS POINT)
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

Include the above functions in the `MatrixFactorization` class.

In [17]:
MatrixFactorization.fit_sgd = fit_sgd
MatrixFactorization.fit = fit

In [ ]:
if DEBUG:   
    print("model without baseline")
    mf.w_average = False
    mf.w_biases = False
    mf.fit(X, y, n_users, n_items)

    print()
    print("after training:")
    print("mf.P_ =", mf.P_)
    print("mf.Q_ =", mf.Q_)
    
    print()
    print("model with baseline")
    mf.w_average = True
    mf.w_biases = True
    mf.fit(X, y, n_users, n_items)

    print()
    print("after training:")
    print("mf.P_ =", mf.P_)
    print("mf.Q_ =", mf.Q_)

**DEBUG:** The previous cell should output:

```
model without baseline
{'eta': 0.002, 'k': 5, 'lam': 0.0, 'n_epochs': 5, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1, 'w_average': False, 'w_biases': False}
start of training
epoch 0 loss 13.639409087626682
epoch 1 loss 13.230478844754824
epoch 2 loss 7.0618821126408635
epoch 3 loss 2.725228011126733
epoch 4 loss 1.7016379313310284

after training:
mf.P_ = [[-0.1698 -0.0308 -0.1026  0.3224 -0.5999]
 [-0.4869  0.194  -0.1285  0.365  -0.5793]
 [-0.2857 -0.16   -0.1121  0.4718 -0.7441]
 ...
 [-0.2098 -0.0308 -0.0361  0.3639 -0.7422]
 [-0.2596  0.1421 -0.0857  0.2301 -0.5589]
 [-0.2356  0.0529 -0.046   0.2977 -0.6761]]
mf.Q_ = [[-1.5851  0.2133 -0.317   2.2759 -3.9413]
 [-1.1774  0.3314 -0.2151  1.7553 -3.2064]
 [-1.1776  0.1    -0.3964  1.6477 -3.046 ]
 ...
 [-0.0474 -0.0531 -0.1062  0.1721 -0.248 ]
 [-0.1908 -0.0122 -0.1645  0.3567 -0.5713]
 [-0.9779  0.0391  0.024   0.967  -2.0613]]

model with baseline
{'eta': 0.002, 'k': 5, 'lam': 0.0, 'n_epochs': 5, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1, 'w_average': True, 'w_biases': True}
start of training
epoch 0 loss 0.9467098876852351
epoch 1 loss 0.8574775878207676
epoch 2 loss 0.8225046228603675
epoch 3 loss 0.800529367369865
epoch 4 loss 0.7850515429727457

after training:
mf.P_ = [[ 0.1623 -0.0605 -0.0533 -0.1063  0.0838]
 [-0.2416  0.1707 -0.0822  0.0332 -0.0291]
 [ 0.1433 -0.2071 -0.0311 -0.042   0.1165]
 ...
 [ 0.0857 -0.0634 -0.0119 -0.0227 -0.0034]
 [-0.0315  0.1172 -0.0372 -0.063  -0.0469]
 [ 0.0272  0.0181  0.0175 -0.0461 -0.0552]]
mf.Q_ = [[ 0.0207 -0.1148  0.0344 -0.0504  0.0295]
 [ 0.0193  0.1361  0.1138  0.1442 -0.0676]
 [ 0.0439 -0.07   -0.1378  0.034  -0.0398]
 ...
 [ 0.0567 -0.0603 -0.1154  0.0364 -0.0378]
 [ 0.0622  0.0091 -0.1184  0.0226  0.0074]
 [-0.1067 -0.0216  0.1438 -0.1057 -0.1115]]
```

### Make Predictions --- TO EDIT

The `predict` method takes as input a set of input user, item pairs in the form of an array `X` (not necessarily the same as that used for training), and returns the predicted ratings as an array `y_pred`.

**TASK:** For each user, item pair in `X`:
- predict a rating and store it into `y_pred`.

In [18]:
def predict(self, X, y=None):
    try:
        getattr(self, "n_users_")
    except AttributeError:
        raise RuntimeError("You must train before predicting!")
    
    
    U, I = to_UI_arrays(X, self.n_users_, self.n_items_)
    
    y_pred = np.ndarray(U.shape[0])
    
    for index in range(U.shape[0]):
        u = U[index]
        i = I[index]
        
        # YOUR CODE HERE
        prediction = self.P_[u].T.dot(self.Q_[i])
        if self.w_average:
            prediction += self.mu_
        if self.w_biases:
            prediction += self.bu_[u] + self.bi_[i]
        
        y_pred[index] = prediction

        
    if y is not None:
        mse = mean_squared_error(y_pred, y)
        rmse = math.sqrt(mse)
        print("RMSE", rmse)
        
    return y_pred


Include it in the `MatrixFactorization` class.

In [19]:
MatrixFactorization.predict = predict

In [20]:
if DEBUG:
    print(mf.predict(X, y))

RMSE 1.0597995576133503
[3.5833 3.4484 3.6245 ... 3.3996 3.462  3.4427]


**DEBUG:** The previous cell should output:

```
RMSE 0.8809997299567512
[3.2295 4.0322 3.8601 ... 3.4563 4.213  3.399 ]
```

## Evaluation

Below are two functions used to evaluate the effectiveness of the recommender.

In [21]:
def computeRMSE(self):
    y_pred = self.predict(self.X_)
    mse = mean_squared_error(y_pred, self.y_)
    rmse = math.sqrt(mse)
    return rmse, mse

def score(self, X, y=None):
    if y is None:
        rmse, mse = self.computeRMSE()
        return -rmse
    else:
        y_pred = self.predict(X)
        mse = mean_squared_error(y_pred, y)
        return -math.sqrt(mse)

Also include them in the `MatrixFactorization` class.

In [22]:
MatrixFactorization.computeRMSE = computeRMSE
MatrixFactorization.score = score

## Additional Tests

In [23]:
MatrixFactorization.predict = predict
MatrixFactorization.fit_init = fit_init
MatrixFactorization.fit_sgd = fit_sgd
MatrixFactorization.fit_mgd = fit_mgd
MatrixFactorization.fit = fit

In [ ]:
mf = MatrixFactorization()
mf.k = 10
mf.lam = 0.001
mf.eta = 0.002
mf.n_epochs = 10
mf.s_batch = 1
mf.w_average = True
mf.w_biases = True

%time mf.fit(X, y, n_users, n_items)

y_pred = mf.predict(X,y)
print("y_pred", y_pred)
print("y", y)

**DEBUG**: The previous cell should output:
```
{'eta': 0.002, 'k': 10, 'lam': 0.001, 'n_epochs': 10, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1, 'w_average': True, 'w_biases': True}
start of training
epoch 0 loss 0.9507106291397593
epoch 1 loss 0.8590495610491214
epoch 2 loss 0.8232773658803088
epoch 3 loss 0.8007329782739687
epoch 4 loss 0.7847901829593874
epoch 5 loss 0.7727912055338106
epoch 6 loss 0.7633699654686578
epoch 7 loss 0.7557260695103907
epoch 8 loss 0.7493489153943951
epoch 9 loss 0.743891555592705
CPU times: user 1min 17s, sys: 790 ms, total: 1min 18s
Wall time: 1min 19s
RMSE 0.8594944026423732
y_pred [3.1841 3.8918 3.8298 ... 3.5132 4.2292 3.4886]
y [3.5 3.5 3.5 ... 3.5 4.  5. ]
```

In [ ]:
mf = MatrixFactorization()
mf.k = 10
mf.lam = 0.001
mf.eta = 0.002
mf.n_epochs = 10
mf.s_batch = 1
mf.w_average = True
mf.w_biases = False

%time mf.fit(X, y, n_users, n_items)

y_pred = mf.predict(X,y)
print("y_pred", y_pred)
print("y", y)

**DEBUG**: The previous cell should output:
```
{'eta': 0.002, 'k': 10, 'lam': 0.001, 'n_epochs': 10, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1, 'w_average': True, 'w_biases': False}
start of training
epoch 0 loss 1.1209703847886336
epoch 1 loss 1.120067412319799
epoch 2 loss 1.119166258088919
epoch 3 loss 1.1181743824848953
epoch 4 loss 1.1169609077367804
epoch 5 loss 1.115309437605747
epoch 6 loss 1.112838802107659
epoch 7 loss 1.1088688767474086
epoch 8 loss 1.1022134865988296
epoch 9 loss 1.090939129152342
CPU times: user 1min 6s, sys: 704 ms, total: 1min 7s
Wall time: 1min 7s
RMSE 1.0402325452357721
y_pred [3.5057 3.5041 3.6061 ... 3.5149 3.4733 3.5145]
y [3.5 3.5 3.5 ... 3.5 4.  5. ]
```

## BONUS POINT

If you additionally implement the following method, you will be awarded a bonus point --- and you will also feel proud that you master vectorization!

- Implement mini-batch stochastic gradient descent in a vectorized manner. You should implement method `fit_mgd` and include it in the class.


The key is to use the `X` matrix. You may have to split it into a user matrix `X_U` and an item matrix `X_I` part. Then, use these matrices to repeat the appropriate rows from matrices `P` and `Q`. That is, you should create a matrix `PB` where its k-th row is the row of `P` corresponding to the user indicated at the k-th row of matrix `X_U`. Similarly you create a matrix `QB`. In the end, you should compute the inner product of each row of `PB` with the corresponding row of `QB`.


Optionally, you may also implement `predict` in the similar vectorized manner.

In [ ]:
mf = MatrixFactorization()
mf.k = 10
mf.lam = 0.001
mf.eta = 0.002
mf.n_epochs = 10
mf.s_batch = 1000
mf.w_average = True
mf.w_biases = False

%time mf.fit(X, y, n_users, n_items)

y_pred = mf.predict(X,y)
print("y_pred", y_pred)
print("y", y)

{'eta': 0.002, 'k': 10, 'lam': 0.001, 'n_epochs': 10, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1000, 'w_average': True, 'w_biases': False}
start training
9924 968
X_ (394638, 10892)
self.P_ (9924, 10)


**Expected output:**
Derived from truncated data ```DEBUG = True```
```
{'eta': 0.002, 'k': 10, 'lam': 0.001, 'n_epochs': 10, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1000, 'w_average': True, 'w_biases': False}
start training
9924 968
X_ (394638, 10892)
self.P_ (9924, 10)
epoch 0 loss 1.1191333723537802
epoch 1 loss 1.118227175637557
epoch 2 loss 1.1173353788004845
epoch 3 loss 1.1163666902740568
epoch 4 loss 1.1151964757937785
epoch 5 loss 1.1136231861155976
epoch 6 loss 1.1112967899300752
epoch 7 loss 1.1075985843538718
epoch 8 loss 1.1014545613386342
epoch 9 loss 1.0911077547189652
CPU times: user 3.06 s, sys: 82.2 ms, total: 3.14 s
Wall time: 3.18 s
RMSE 1.041532740797242
y_pred [3.5062 3.5048 3.6021 ... 3.5187 3.4791 3.5141]
y [3.5 3.5 3.5 ... 3.5 4.  5. ]
```

In [ ]:
mf = MatrixFactorization()
mf.k = 10
mf.lam = 0.001
mf.eta = 0.002
mf.n_epochs = 10
mf.s_batch = 1000
mf.w_average = True
mf.w_biases = True

%time mf.fit(X, y, n_users, n_items)

y_pred = mf.predict(X,y)
print("y_pred", y_pred)
print("y", y)

**Expected output:**
```
{'eta': 0.002, 'k': 10, 'lam': 0.001, 'n_epochs': 10, 'rnd_mean': 0, 'rnd_std': 0.1, 's_batch': 1000, 'w_average': True, 'w_biases': True}
start training
9924 968
X_ (394638, 10892)
self.P_ (9924, 10)
epoch 0 loss 0.9494371432229923
epoch 1 loss 0.8577499223785994
epoch 2 loss 0.8220173488238905
epoch 3 loss 0.7995001533886877
epoch 4 loss 0.7835805700586336
epoch 5 loss 0.77160479264564
epoch 6 loss 0.7622090723884897
epoch 7 loss 0.7545948294068388
epoch 8 loss 0.7482532026351049
epoch 9 loss 0.7428391454192339
CPU times: user 3.5 s, sys: 70.4 ms, total: 3.57 s
Wall time: 3.59 s
RMSE 0.859641613813476
y_pred [3.1803 3.8931 3.8339 ... 3.5298 4.2333 3.4882]
y [3.5 3.5 3.5 ... 3.5 4.  5. ]
```

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [ ]:
# feel free to use this field for additional tests

In [2]:
# feel free to use this field for additional tests